**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 4.6MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Flatten, Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

# MiniProject on Deep Reinforcement Learning

$\DeclareMathOperator*{\argmax}{argmax}$
$\DeclareMathOperator*{\argmin}{argmin}$

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}\left[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The act function makes an action in the environnement depending on the phase we're in. In the training phase, we perform a random action with an $\epsilon$ probability to do exploration and perform an action with the learned policy with a probability of $ 1- \epsilon$ to evaluate our current policy. $\epsilon$ is meant to avoid getting stuck in local reward optimas.
In the test phase, the agent performs the best action he learned for the current state.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 50 # set small when debugging
epochs_test = 20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array board defines the spatial distribution of the rewards on the $N\times N$ grid. It stores the reward of getting to a state (x,y) in the board. We set the reward to 0 after its collection (the state have already been visited). Red cells lead to positive rewards, blue cells negative rewards and a white cell represents the rat.

The array position is a static array that defines the borders (forbidden states) of the grid by a value of -1, the allowed states by 0 and the position of the agent that is defined by a value of 1.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.0/13.0. Average score (-7.0)
Win/lose count 8.0/9.0. Average score (-4.0)
Win/lose count 16.5/17.0. Average score (-2.8333333333333335)
Win/lose count 4.5/14.0. Average score (-4.5)
Win/lose count 7.5/15.0. Average score (-5.1)
Win/lose count 12.5/12.0. Average score (-4.166666666666667)
Win/lose count 13.0/15.0. Average score (-3.857142857142857)
Win/lose count 8.5/12.0. Average score (-3.8125)
Win/lose count 10.5/12.0. Average score (-3.5555555555555554)
Win/lose count 9.0/8.0. Average score (-3.1)
Win/lose count 10.5/15.0. Average score (-3.227272727272727)
Win/lose count 8.0/17.0. Average score (-3.7083333333333335)
Win/lose count 9.0/18.0. Average score (-4.115384615384615)
Win/lose count 5.5/12.0. Average score (-4.285714285714286)
Win/lose count 9.5/12.0. Average score (-4.166666666666667)
Win/lose count 10.5/19.0. Average score (-4.4375)
Win/lose count 11.5/14.0. Average score (-4.323529411764706)
Win/lose count 5.5/9.0. Average score (-4.277777777777778)
Win/l

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{align*}Q^\pi(s,a)= E_{p^{\pi}}\left[r(s_0,a_0)|s_{0}=s,a_{0}=a\right] + E_{p^{\pi}}\left[\sum_{1 \leq t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \> .
\end{align*}

Here  $(s_0, a_0)$ does not depend on the policy but only on $p$, hence:

\begin{align*}Q^\pi(s,a) &= E \left[r(s_0,a_0)\right] + \gamma E_{p^{\pi}}\left[\sum_{1 \leq t\leq T}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \> \\
& = E \left[r(s_0,a_0)\right] + \gamma \sum_{s',a'} p((s', a')|s_{0}=s,a_{0}=a) E_{p^{\pi}}\left[\sum_{1 \leq t\leq T} \gamma^{t-1}r(s_{t},a_{t})|s_{1}=s', a_{1} = a'\right] .\\
& = E \left[r(s_0,a_0)\right] + \gamma \sum_{s', a'} p((s', a')|s_{0}=s,a_{0}=a) E_{p^{\pi}}\left[\sum_{t'\leq T} \gamma^{t'}r(s_{t'},a_{t'})|s_{1-1}=s', a_{1-1}=a'\right] \\
& = E \left[r(s_0,a_0)\right] + \gamma E_{(s',a')\sim p(.|s,a)}\left[E_{p^{\pi}}\left[\sum_{t'\leq T} \gamma^{t'}r(s_{t'},a_{t'})|s_{1-1}=s', a_{1-1}=a'\right] \right] \\
Q^{\pi}(s,a) &= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}

$$\boxed{
\begin{equation*}
Q^{\pi}(s,a) = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \> .
\end{equation*}
}$$

because $(s,a)$ does not depend on the next state $(s',a')$.

\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

From the definition above one can notice that $p((s',a')|(s,a)) = \pi(a'|s') p(s'|(a,s))$

Maimizing over $\pi$ means choosing for a given state $s'$ the best action $a'$.
So, given its existence:

\begin{equation*}
\pi^*(s'|(s,a))= \argmax_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{\pi}(s',a')] \> .
\end{equation*}

So, under $\pi^*$, the next state $s'$ is drawn with respect to $p(.,(s,a))$. For sake of consistency with the notations, 

$$\boxed{
\begin{align*}
Q^*(s,a) &= \max_\pi E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
Q^{*}(s,a) &= E_{s' \sim \pi^*(.|s,a)}[\max_{a'}r(s,a) + \gamma \max_{a'} Q^{*}(s',a')]
\end{align*}
}$$

$Q^*$ is the solution of $F(Q) = 0$ with 

\begin{equation*}
F(Q)= E_{s' \sim \pi^*(.|s,a)}[\max_{a'}r(s,a) + \gamma \max_{a'} Q(s',a') - Q(s,a)] \> .
\end{equation*}

So a plausible loss that aims at minimizing the residuals of that equation can be:

$$\boxed{
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma \max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}
}$$

with $\theta$ the parameters of the neural network.




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            #take a random batch from our buffer
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            #store the current state and generate its state-action function
            input_states[i] = s_
            target_q[i] = self.model.predict(s_[None])
            if game_over_:
               target_q[i, a_] = r_
            else:
                ns_q = self.model.predict(n_s_[None])
                target_q[i, a_] = r_ + self.discount * ns_q.max(1)
        ######## FILL IN
        # HINT: Clip the target to avoid exploding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state, )))
        model.add(Dense(units=32, activation= 'relu'))
        model.add(Dense(units=4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))













Epoch 000/050 | Loss 0.0058 | Win/lose count 2.5/5.0 (-2.5)
Epoch 001/050 | Loss 0.0216 | Win/lose count 4.0/5.0 (-1.0)
Epoch 002/050 | Loss 0.0154 | Win/lose count 2.5/4.0 (-1.5)
Epoch 003/050 | Loss 0.0170 | Win/lose count 2.0/6.0 (-4.0)
Epoch 004/050 | Loss 0.0089 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/050 | Loss 0.0171 | Win/lose count 2.5/6.0 (-3.5)
Epoch 006/050 | Loss 0.0106 | Win/lose count 1.0/3.0 (-2.0)
Epoch 007/050 | Loss 0.0585 | Win/lose count 1.5/3.0 (-1.5)
Epoch 008/050 | Loss 0.0046 | Win/lose count 2.5/3.0 (-0.5)
Epoch 009/050 | Loss 0.0615 | Win/lose count 3.5/1.0 (2.5)
Epoch 010/050 | Loss 0.0082 | Win/lose count 4.5/2.0 (2.5)
Epoch 011/050 | Loss 0.0125 | Win/lose count 1.0/1.0 (0.0)
Epoch 012/050 | Loss 0.0130 | Win/lose count 2.0/3.0 (-1.0)
Epoch 013/050 | Loss 0.0020 | Win/lose count 0.5/7.0 (-6.5)
Epoch 014/050 | Loss 0.0024 | Win/lose count 3.5/0 (3.5)
Epoch 015/050 | Loss 0.0086 | Win/lose count 4.0/4.0 (0.0)
Epoch 016/050 | Loss 0.0034 | Win/lo

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(5, 5, self.n_state,)))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(units = 4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0006 | Win/lose count 7.5/7.0 (0.5)
Epoch 001/050 | Loss 0.0034 | Win/lose count 6.5/6.0 (0.5)
Epoch 002/050 | Loss 0.0072 | Win/lose count 8.5/1.0 (7.5)
Epoch 003/050 | Loss 0.0600 | Win/lose count 7.0/2.0 (5.0)
Epoch 004/050 | Loss 0.0957 | Win/lose count 4.0/4.0 (0.0)
Epoch 005/050 | Loss 0.0017 | Win/lose count 6.0/5.0 (1.0)
Epoch 006/050 | Loss 0.0034 | Win/lose count 3.5/2.0 (1.5)
Epoch 007/050 | Loss 0.0020 | Win/lose count 6.5/2.0 (4.5)
Epoch 008/050 | Loss 0.0028 | Win/lose count 6.5/2.0 (4.5)
Epoch 009/050 | Loss 0.0025 | Win/lose count 5.0/3.0 (2.0)
Epoch 010/050 | Loss 0.0039 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/050 | Loss 0.0021 | Win/lose count 4.0/0 (4.0)
Epoch 012/050 | Loss 0.0027 | Win/lose count 5.0/6.0 (-1.0)
Epoch 013/050 | Loss 0.0048 | Win/lose count 1.0/2.0 (-1.0)
Epoch 014/050 | Loss 0.0023 | Win/lose count 13.0/3.0 (10.0)
Epoch 015/050 | Loss 0.0485 | Win/lose count 2.0/2.0 (0.0)
Epoch 016/050 | Loss 0.0028 | Win/lose count 5.0/3.0 (

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 8.5/3.0. Average score (5.5)
Win/lose count 10.0/5.0. Average score (5.25)
Win/lose count 7.0/5.0. Average score (4.166666666666667)
Win/lose count 4.5/1.0. Average score (4.0)
Win/lose count 7.0/3.0. Average score (4.0)
Win/lose count 4.5/1.0. Average score (3.9166666666666665)
Win/lose count 10.0/3.0. Average score (4.357142857142857)
Win/lose count 3.5/2.0. Average score (4.0)
Win/lose count 6.0/5.0. Average score (3.6666666666666665)
Win/lose count 8.0/4.0. Average score (3.7)
Win/lose count 4.5/2.0. Average score (3.590909090909091)
Win/lose count 6.5/4.0. Average score (3.5)
Win/lose count 11.0/1.0. Average score (4.0)
Win/lose count 8.0/6.0. Average score (3.857142857142857)
Win/lose count 8.5/2.0. Average score (4.033333333333333)
Win/lose count 13.5/3.0. Average score (4.4375)
Win/lose count 3.0/4.0. Average score (4.117647058823529)
Win/lose count 7.5/0. Average score (4.305555555555555)
Win/lose count 5.0/2.0. Average score (4.2368421052631575)

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')











Test of the CNN
Win/lose count 4.0/1.0. Average score (3.0)
Win/lose count 3.0/1.0. Average score (2.5)
Win/lose count 3.5/0. Average score (2.8333333333333335)
Win/lose count 2.5/1.0. Average score (2.5)
Win/lose count 1.0/1.0. Average score (2.0)
Win/lose count 2.5/0. Average score (2.0833333333333335)
Win/lose count 5.5/0. Average score (2.5714285714285716)
Win/lose count 3.0/5.0. Average score (2.0)
Win/lose count 1.5/1.0. Average score (1.8333333333333333)
Win/lose count 3.0/0. Average score (1.95)
Win/lose count 0.5/0. Average score (1.8181818181818181)
Win/lose count 4.0/0. Average score (2.0)
Win/lose count 0/0. Average score (1.8461538461538463)
Win/lose count 4.0/4.0. Average score (1.7142857142857142)
Win/lose count 2.5/4.0. Average score (1.5)
Win/lose count 2.5/2.0. Average score (1.4375)
Win/lose count 2.0/1.0. Average score (1.411764705882353)
Win/lose count 1.5/1.0. Average score (1.3611111111111112)
Win/lose count 0.5/2.0. Average score (1.2105263157894737)
W

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 20.0/1.0. Average score (19.0)
Win/lose count 15.5/2.0. Average score (16.25)
Win/lose count 15.0/4.0. Average score (14.5)
Win/lose count 6.0/0. Average score (12.375)
Win/lose count 20.0/3.0. Average score (13.3)
Win/lose count 10.0/4.0. Average score (12.083333333333334)
Win/lose count 14.0/1.0. Average score (12.214285714285714)
Win/lose count 21.0/2.0. Average score (13.0625)
Win/lose count 15.0/6.0. Average score (12.61111111111111)
Win/lose count 12.5/2.0. Average score (12.4)
Win/lose count 20.5/3.0. Average score (12.863636363636363)
Win/lose count 12.0/1.0. Average score (12.708333333333334)
Win/lose count 12.0/0. Average score (12.653846153846153)
Win/lose count 19.0/1.0. Average score (13.035714285714286)
Win/lose count 17.5/3.0. Average score (13.133333333333333)
Win/lose count 11.5/2.0. Average score (12.90625)
Win/lose count 17.0/3.0. Average score (12.970588235294118)
Win/lose count 16.0/3.0. Average score (12.972222222222221)
Win/lose cou

The temperature accounts for the the proportion of malus and bonus. Wen the temperature is smaller, they are less filled cases, so the agents tend to explore a little bit more resulting in better results.


The method with CNN outperforms fully-connected DQN by more than $5$ points. Indeed, we win with DQN_CNN for a standard temperature of $0.3$. For a smaller temperature, the gap of performance between both approaches is smaller, while both approaches perform really well for a bigger temparature of 0.8. We can notice that training for a smaller temperature helps for the bigger one.


When we watch the video, we can notice that both methods do not explore enough the space. Indeed, for quite a few episodes, the rat remains stuck, going back and forth between a position A and a position B. Enhancing exploration can help improving the method.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    #decay 0.3 to approximately 0.1 for each epoch
    init_eps = agent.epsilon
    B = 0.1
    A = init_eps - B
    T = 5.0 / epoch
    # decay_parameter_epsilon = 0.1
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        eps = A * np.exp(-e/T) + B
        agent.set_epsilon(eps)
        # agent.set_epsilon(max(agent.epsilon*(1-decay_parameter_epsilon), 0.1))
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # RL agent
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

            # Apply the reinfo
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus > 0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        # beginning malus set to 0
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


    
## use those samples of code:
#In train explore:
# state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [0]:
# Training
epochs_train = 50
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))













Epoch 000/050 | Loss 0.0143 | Win/lose count 12.5/22.700000000000042 (-10.200000000000042)
Epoch 001/050 | Loss 0.0047 | Win/lose count 5.0/21.50000000000001 (-16.50000000000001)
Epoch 002/050 | Loss 0.0084 | Win/lose count 11.0/21.200000000000053 (-10.200000000000053)
Epoch 003/050 | Loss 0.0681 | Win/lose count 13.0/17.799999999999994 (-4.799999999999994)
Epoch 004/050 | Loss 0.0134 | Win/lose count 4.5/21.00000000000004 (-16.50000000000004)
Epoch 005/050 | Loss 0.0067 | Win/lose count 4.0/20.900000000000038 (-16.900000000000038)
Epoch 006/050 | Loss 0.0075 | Win/lose count 11.0/16.59999999999997 (-5.5999999999999694)
Epoch 007/050 | Loss 0.0073 | Win/lose count 4.5/18.900000000000006 (-14.400000000000006)
Epoch 008/050 | Loss 0.0061 | Win/lose count 17.0/15.09999999999997 (1.9000000000000306)
Epoch 009/050 | Loss 0.0060 | Win/lose count 20.0/15.499999999999964 (4.5000000000000355)
Epoch 010/050 | Loss 0.0062 | Win/lose count 16.0/15.199999999999969 (0.8000000000000309)
E

In [0]:
# Evaluation
epochs_test = 30 # set small when debugging
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 19.0/15.099999999999966. Average score (3.900000000000034)
Win/lose count 17.5/16.29999999999998. Average score (2.5500000000000274)
Win/lose count 19.0/14.199999999999973. Average score (3.3000000000000274)
Win/lose count 24.5/13.699999999999976. Average score (5.175000000000027)
Win/lose count 21.5/16.399999999999984. Average score (5.160000000000025)
Win/lose count 21.0/18.40000000000001. Average score (4.733333333333353)
Win/lose count 21.0/15.499999999999968. Average score (4.842857142857164)
Win/lose count 21.5/13.89999999999997. Average score (5.187500000000022)
Win/lose count 25.0/13.79999999999997. Average score (5.855555555555578)
Win/lose count 19.5/13.99999999999997. Average score (5.820000000000023)
Win/lose count 20.5/13.599999999999975. Average score (5.918181818181841)
Win/lose count 20.5/14.699999999999969. Average score (5.908333333333356)
Win/lose count 18.5/12.399999999999972. Average score (5.923076923076946)
Win/lose count 23.5/13.79999999999998. Av

It seems that decreasing exponentially $\epsilon$ works better. It also outputs satisfying moves in the videos, but the performance can be a bit lower than for the previous approach, but after a few trials, it seemed to work slightly better than the previous approaches because of the degree of freedom. However, the agent doesn't memorize information on the rewards in the areas it has visited, only the fact that it visited them, so it can lose choosing not to backtrack to areas with remaining cheese.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***

In [0]:
class DQN_Mim(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_Mim, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_[:,:,1:], a_]) #now we will not use the fact that our model need to verify the bellman equation but directly copy what the expert has done
        #hence we juste need the current state and the action that has been applied
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 3  #Now we want to mimick the expert hence we want our target function to be maximized 
            #in the action applied by the expert
            
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_CNN_Mim(DQN_Mim):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_Mim, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, kernel_size=3, input_shape=(5,5,self.n_state,),activation='relu')) #model with 1 hidden CNN performs the best
        #with my method from what I ve experimented
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
def train_mimick(expert,agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = expert.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [0]:
epochs_train = 100
expert = agent
agent_n = DQN_CNN_Mim(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimick(expert,agent_n, env, epochs_train, prefix='cnn_train_mimick')

Epoch 000/100 | Loss 0.0391 | Win/lose count 7.5/18.29999999999999 (-10.79999999999999)
Epoch 001/100 | Loss 0.0051 | Win/lose count 2.5/19.300000000000004 (-16.800000000000004)
Epoch 002/100 | Loss 0.1828 | Win/lose count 17.5/13.899999999999967 (3.6000000000000334)
Epoch 003/100 | Loss 0.1864 | Win/lose count 12.0/16.499999999999964 (-4.4999999999999645)
Epoch 004/100 | Loss 0.1863 | Win/lose count 12.0/16.499999999999964 (-4.4999999999999645)
Epoch 005/100 | Loss 0.2318 | Win/lose count 17.5/13.399999999999968 (4.100000000000032)
Epoch 006/100 | Loss 0.3646 | Win/lose count 24.0/9.99999999999998 (14.00000000000002)
Epoch 007/100 | Loss 0.2927 | Win/lose count 19.0/12.599999999999971 (6.400000000000029)
Epoch 008/100 | Loss 0.2350 | Win/lose count 17.5/15.199999999999962 (2.300000000000038)
Epoch 009/100 | Loss 0.1669 | Win/lose count 15.0/14.599999999999964 (0.4000000000000359)
Epoch 010/100 | Loss 0.2765 | Win/lose count 10.0/16.09999999999996 (-6.099999999999959)
Epoch 011/100 | L

In [0]:
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_n, env_test, epochs_test,prefix='cnn_test_mimick')

Win/lose count 12.0/3.0. Average score (9.0)
Win/lose count 17.5/10.0. Average score (8.25)
Win/lose count 15.0/3.0. Average score (9.5)
Win/lose count 19.0/2.0. Average score (11.375)
Win/lose count 20.5/4.0. Average score (12.4)
Win/lose count 20.0/3.0. Average score (13.166666666666666)
Win/lose count 9.0/6.0. Average score (11.714285714285714)
Win/lose count 12.5/3.0. Average score (11.4375)
Win/lose count 15.0/2.0. Average score (11.61111111111111)
Win/lose count 7.0/1.0. Average score (11.05)
Win/lose count 7.0/2.0. Average score (10.5)
Win/lose count 14.0/3.0. Average score (10.541666666666666)
Win/lose count 19.0/3.0. Average score (10.961538461538462)
Win/lose count 18.5/5.0. Average score (11.142857142857142)
Win/lose count 19.5/5.0. Average score (11.366666666666667)
Win/lose count 6.0/1.0. Average score (10.96875)
Win/lose count 20.0/7.0. Average score (11.088235294117647)
Win/lose count 11.0/2.0. Average score (10.972222222222221)
Win/lose count 16.0/4.0. Average score (11

In [0]:
HTML(display_videos('cnn_test_mimick10.mp4'))

It performs even better than previously quite suprisingly. Indeed, it does take less time at each epoch, so we can afford to extend the number of epochs. The results are better for an equivalent amount of time. The results have high variance though. 


All in all, based on various experiments, it seems that all methods with CNN work, but have some variance. It is even more the case for the latest method, but it allows to find schemes that can work better for testing. 